In [ ]:
import pandas as pd
import numpy as np


In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import collections #for frequeny counting
import findspark
findspark.init("/home/spjain/spark2/")

import pyspark
from pyspark.sql import DataFrameNaFunctions
from pyspark.sql.functions import lit #create columns of *literal* value
from pyspark.sql.functions import col #Returns columns given on column name
from pyspark.ml.feature import StringIndexer #label encoding
from pyspark.ml import Pipeline

sc = pyspark.SparkContext(appName="demonitisation")

In [2]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import re
import seaborn as sns
from IPython.display import display
pd.options.mode.chained_assignment = None
import matplotlib
matplotlib.style.use('ggplot')

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Read CSV file

In [54]:
tweets = spark.read.csv('data/demonetization-tweets.csv', encoding = "ISO-8859-1", header="true")
type(tweets)
tweets.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- text: string (nullable = true)
 |-- favorited: string (nullable = true)
 |-- favoriteCount: string (nullable = true)
 |-- replyToSN: string (nullable = true)
 |-- created: string (nullable = true)
 |-- truncated: string (nullable = true)
 |-- replyToSID: string (nullable = true)
 |-- id: string (nullable = true)
 |-- replyToUID: string (nullable = true)
 |-- statusSource: string (nullable = true)
 |-- screenName: string (nullable = true)
 |-- retweetCount: string (nullable = true)
 |-- isRetweet: string (nullable = true)
 |-- retweeted: string (nullable = true)



In [55]:

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

## create user defined function to extract title
gettweetos = udf(lambda text: text.split(':')[0].strip() if text.find('RT @') > -1 else 'other', StringType())
getnewtext = udf(lambda text: re.search('(?<=:)(.*)', text).group(0) if text.find('RT @') > -1 else 'other', StringType())
tweets = tweets.withColumn('tweetos', gettweetos(tweets['text']))
tweets = tweets.withColumn('text_new', getnewtext(tweets['text']))


In [56]:
tweets.select('text','tweetos', 'text_new').show(5)

+--------------------+-----------------+--------------------+
|                text|          tweetos|            text_new|
+--------------------+-----------------+--------------------+
|RT @rssurjewala: ...|  RT @rssurjewala| Critical questio...|
|RT @Hemant_80: Di...|    RT @Hemant_80| Did you vote on ...|
|RT @roshankar: Fo...|    RT @roshankar| Former FinSec, R...|
|RT @ANI_news: Gur...|     RT @ANI_news| Gurugram (Haryan...|
|RT @satishacharya...|RT @satishacharya| Reddy Wedding! @...|
+--------------------+-----------------+--------------------+
only showing top 5 rows



0